In [2]:
%load_ext autoreload
%autoreload 2
import os
# os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = ".99"
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# import pandas as pd
import dcmnet
import sys

sys.path.append("/home/boittier/jaxeq/dcmnet")
# print(sys.path)
from dcmnet.models import DCM1, DCM2, DCM3, DCM4, dcm1_params, dcm2_params, dcm3_params, dcm4_params
from dcmnet.modules import MessagePassingModel
from dcmnet.data import prepare_datasets

import numpy as np

def atom_centered_dipole(dcm, com, q):
    dipole_out = np.zeros(3)
    for i, _ in enumerate(dcm):
        dipole_out += q[i] * (_ - com)
    # print(dipole_out*2.5417464519)
    return np.linalg.norm(dipole_out)* 4.80320

import jax
import jax.numpy as jnp
import pickle
from tensorboardX import SummaryWriter
import time
from dcmnet.utils import safe_mkdir
from dcmnet.training import train_model
from dcmnet.training_dipole import train_model_dipo
from pathlib import Path
from dcmnet.data import prepare_batches, prepare_datasets
from dcmnet.utils import apply_model
from dcmnet.plotting import evaluate_dc, plot_esp, plot_model
import optax
from dcmnet.loss import esp_mono_loss_pots
from dcmnet.analysis import create_model_and_params
import matplotlib.pyplot as plt
from tqdm import tqdm
import random

devices = jax.local_devices()
print(devices)
print(jax.default_backend())
print(jax.devices())
from dcmnet.utils import apply_model, clip_colors, reshape_dipole
from dcmnet.data import prepare_batches, cut_vdw


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[CpuDevice(id=0)]
cpu
[CpuDevice(id=0)]


<Figure size 640x480 with 0 Axes>

<Figure size 100x100 with 0 Axes>

In [29]:
from dcmnet.analysis import *
from tqdm import tqdm

path = Path("/pchem-data/meuwly/boittier/home/jaxeq/misc")
paths = list(path.glob("*"))
print(paths[0])
# for path in tqdm(paths[:2]):
multipoles(paths[9])

/pchem-data/meuwly/boittier/home/jaxeq/misc/dsgdb9nsd_128167


{'mono': array([-6.89695072e-03, -6.82634603e-03, -6.08240419e-03, ...,
         1.09206025e-09,  1.09206025e-09,  1.09206025e-09]),
 'dipo': array([-7.29053686e-03, -7.40661290e-03, -6.89535156e-03, ...,
         1.17315690e-09,  1.17315690e-09,  1.17315690e-09]),
 'quad': array([-7.27839981e-03, -7.35752800e-03, -6.79966470e-03, ...,
         1.17315828e-09,  1.17315828e-09,  1.17315828e-09]),
 'esp': array([-0.00725657, -0.00726441, -0.00661951, ...,  0.        ,
         0.        ,  0.        ]),
 'closest_atom_type': array([7, 7, 7, ..., 1, 1, 1]),
 'closest_atom': array([ 0,  0,  0, ..., 12, 12, 12]),
 'mask': array([ True,  True,  True, ...,  True,  True,  True]),
 'rmse_mono': Array(1.5774539, dtype=float32),
 'rmse_dipo': Array(1.3724021, dtype=float32),
 'rmse_quad': Array(1.2095424, dtype=float32),
 'rmse_mono_masked': Array(0.8994933, dtype=float32),
 'rmse_dipo_masked': Array(0.5674174, dtype=float32),
 'rmse_quad_masked': Array(0.43492696, dtype=float32),
 'D_mono': Arra

In [28]:
[(i, _) for i, _ in enumerate(paths) if "dsgdb9nsd_128954" in str(_)]

[(9,
  PosixPath('/pchem-data/meuwly/boittier/home/jaxeq/misc/dsgdb9nsd_128954'))]

In [33]:
path = Path("/pchem-data/meuwly/boittier/home/jaxeq/misc")
paths = list(path.glob("*"))
# print(paths[0])
for path in tqdm(paths[3392:]):
    multipoles(path)

100%|██████████████████████████████████████████████████████████████████████████████████████████| 3393/3393 [48:20<00:00,  1.17it/s]


In [32]:
len(paths)//2

3392